In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
import os

In [ ]:
## config

train_dir = '/content/drive/MyDrive/train'
val_dir = '/content/drive/MyDrive/valid'

In [ ]:
train_data_generator = ImageDataGenerator(rescale = 1.255,validation_split=0.2)


In [ ]:
directories = os.listdir('/content/drive/MyDrive/train')

In [ ]:
data = train_data_generator.flow_from_directory(train_dir,batch_size=8 , class_mode='categorical',target_size = (256,256),shuffle = True,subset='training')

Found 1505 images belonging to 4 classes.


In [ ]:
val_data = train_data_generator.flow_from_directory(train_dir,batch_size=8 , class_mode='categorical',target_size = (256,256),shuffle = True,subset='validation')

Found 375 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras import Input

In [ ]:
inputShape = Input(shape=(256,256,3))

base_model = ResNet50(include_top=False,input_tensor=inputShape)
base_model.trainable = False

In [ ]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [ ]:
last_layer = base_model.output

In [ ]:
last_layer

<KerasTensor: shape=(None, 8, 8, 2048) dtype=float32 (created by layer 'conv5_block3_out')>

In [ ]:
x = layers.Flatten()(last_layer)
x = layers.Dense(1024,activation="relu")(x)
x = layers.Dense(4,activation='softmax')(x)

custom_model = Model(inputShape,x)

In [ ]:
custom_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
custom_model.compile(optimizer="Adam",loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
custom_model.fit(data,epochs=15,validation_data=val_data)

Epoch 1/15
189/189 [==============================] - 61s 275ms/step - loss: 26.3220 - accuracy: 0.5714 - val_loss: 4.5008 - val_accuracy: 0.7573
Epoch 2/15
189/189 [==============================] - 48s 254ms/step - loss: 4.5132 - accuracy: 0.6645 - val_loss: 4.5478 - val_accuracy: 0.5360
Epoch 3/15
189/189 [==============================] - 47s 251ms/step - loss: 2.2096 - accuracy: 0.6944 - val_loss: 2.1115 - val_accuracy: 0.7120
Epoch 4/15
189/189 [==============================] - 49s 258ms/step - loss: 0.9847 - accuracy: 0.7309 - val_loss: 1.1912 - val_accuracy: 0.4960
Epoch 5/15
189/189 [==============================] - 48s 252ms/step - loss: 0.6142 - accuracy: 0.7754 - val_loss: 1.0726 - val_accuracy: 0.5653
Epoch 6/15
189/189 [==============================] - 49s 258ms/step - loss: 0.5763 - accuracy: 0.7834 - val_loss: 1.5828 - val_accuracy: 0.5040
Epoch 7/15
189/189 [==============================] - 53s 280ms/step - loss: 0.5913 - accuracy: 0.7714 - val_loss: 1.1744 - val_a